# DQN Trading Agent

## Read database

In [1]:
#TO READ THE DATABASE
import h5py

DB_FILE = "../data/dataset_1h_1000.hdf5"
DB = h5py.File(DB_FILE, "r")

for item in DB:
    print(item)

bitcoin_usd
etherium_usd
ripple_usd


In [2]:
# Show the values of the first timestep
print(DB["bitcoin_usd"][0:2])

[[  1.51543080e+12   1.47930000e+04   1.46940000e+04   1.48600000e+04
    1.46690000e+04   7.86847895e+02]
 [  1.51542720e+12   1.44000000e+04   1.47890000e+04   1.49230000e+04
    1.43470000e+04   4.20724394e+03]]


## Define packages

In [3]:
import random
import numpy as np

## Define DQN Agent

In [4]:
from collections import deque
from keras.models import Model 
from keras.layers import Input, Dense, Flatten, Reshape
from keras.optimizers import Adam
from keras import backend as K

K.set_image_dim_ordering('tf')

class DQNAgent:
    def __init__(self, frame_size, action_size):
        self.frame_size = frame_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        inputs = Input(shape=(6,100))
        x = Flatten()(inputs)
        x = Dense(32, activation='relu')(x)
        x = Dense(32, activation='relu')(x)
        x = Dense(16, activation='relu')(x)
        out = Dense(self.action_size, activation='linear')(x)
        
        model = Model(inputs=inputs, outputs=out)
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        print(model.summary())
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # The agent acts randomly
            return random.randrange(self.action_size)
        
        # Predict the reward value based on the given state
        act_values = self.model.predict(state)
        
        # Pick the action based on the predicted reward
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        # Sample minibatch from the memory
        minibatch = random.sample(self.memory, batch_size)
        
        # Extract informations from each memory
        for state, action, reward, next_state, done in minibatch:
            
            # if done, make our target reward
            target = reward
            if not done:
                # predict the future discounted reward
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
                
            # make the agent to approximately map
            # the current state to future discounted reward
            # We'll call that target_f
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            # Train the Neural Net with the state and target_f
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

Using TensorFlow backend.


## Define Trading Game

In [5]:
class TradingGame:
    def __init__(self, start_capital, frame_size, database):
        self.start_capital = start_capital
        self.frame_size = frame_size
        
        self.database = np.asarray(database["bitcoin_usd"])
        self.capital = start_capital
        self.timestep = 0
        self.time_frame = self.database[0:self.frame_size]
        
        self.position = 0
        self.state = None
    
    ### Reset function
    def reset(self):
        # Set everything to the initial start value
        self.capital = self.start_capital
        self.timestep = 0
        self.time_frame = self.database[0:self.frame_size]
        
        self.position = 0
        
        # state => (time_frame, capital, position)
        self.state = self.time_frame #self.capital, self.position)
        
        return self.state
    
    
    ### Next frame function
    def get_next_frame(self):
        self.timestep += 1
        
        if (self.frame_size+self.timestep) < self.database.shape[0]:
            next_frame = self.database[self.timestep:self.frame_size+self.timestep]
            done = False
        else:
            next_frame = self.time_frame
            done = True
            
        return next_frame, done
    
    ### Reward function
    def calc_reward(self, action, next_frame):
        # Get new price
        new_price = next_frame[self.frame_size-1, 2]
        
        # Check the action and calculate the reward
        #hold
        if action == 0: 
            reward = 0
            
        #buy
        elif action == 1: 
            if self.position is 0:
                reward = 0
                self.position = new_price
            else:
                reward = -10
                
        #sell
        elif action == 2: 
            if self.position is not 0:
                reward = new_price - self.position
                self.position = 0
            else:
                reward = -10
        return reward
    
    ### Next step function
    def step(self, action):
        # Get next time frame
        next_frame, done = self.get_next_frame()
        
        # Get reward
        reward = self.calc_reward(action, next_frame)
        
        # check if done
        self.capital += reward
        if self.capital <= 0:
            done = True
            
        # Next state
        next_state = next_frame #self.capital, self.position)
        
        # update state
        self.state = next_state
        
        return next_state, reward, done
    
    def get_score(self):
        return self.capital

## Train the agent

### Set parameters

In [6]:
# Parameter for Agent
frame_size = 100 # Time frame
#state_attributes = 8
action_size = 3 # hold, buy, sell

# Parameter for Trading
batch_size = 128
EPISODES = 1000
CAPITAL = 1000
#POSITION = 0

### Training

In [7]:
# Create agent
agent = DQNAgent(frame_size, action_size)

# Initialize Environment
env = TradingGame(CAPITAL, frame_size, DB)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 6, 100)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                19232     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 51        
Total params: 20,867
Trainable params: 20,867
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
#Episodes to train
for e in range(EPISODES):
    
    #Create initial state from time frame
    state = env.reset()
    state = np.asarray(state)
    state = np.reshape(state, [1, 6, frame_size,])

    #time frames
    for time_frame in range(frame_size, 1000):
        
        #Agent takes action
        action = agent.act(state)

        #Calc reward
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, 6, frame_size])
        
        #Remember action
        agent.remember(state, action, reward, next_state, done)
        
        #Override state with next state
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, env.get_score(), agent.epsilon))
            break
       
    #Replay
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
    # if e % 10 == 0:
    #     agent.save("./save/cartpole-dqn.h5")
        
    

episode: 0/1000, score: -1.097828980000486, e: 1.0
episode: 1/1000, score: -1.9511952700013353, e: 1.0
episode: 2/1000, score: -11.203802529998939, e: 0.99
episode: 3/1000, score: -247.14101024999945, e: 0.99
episode: 4/1000, score: -45.097828980000486, e: 0.99
episode: 5/1000, score: -117.95889014000022, e: 0.98
episode: 6/1000, score: -1261.0978289800005, e: 0.98
episode: 7/1000, score: -21.902171019999514, e: 0.97
episode: 8/1000, score: 4025.887837389997, e: 0.97
episode: 9/1000, score: 9062.94342805, e: 0.96
episode: 10/1000, score: -503.2352294499997, e: 0.96
episode: 11/1000, score: -127.90217101999951, e: 0.95
episode: 12/1000, score: -265.3016315099994, e: 0.95
episode: 13/1000, score: -205.60402869000063, e: 0.94
episode: 14/1000, score: -271.0, e: 0.94
episode: 15/1000, score: -52.018728420001935, e: 0.93
episode: 16/1000, score: -152.23883922999994, e: 0.93
episode: 17/1000, score: -1514.198071750001, e: 0.92
episode: 18/1000, score: -320.0, e: 0.92
episode: 19/1000, score:

episode: 166/1000, score: -437.0, e: 0.44
episode: 167/1000, score: -172.0, e: 0.44
episode: 168/1000, score: -4.0, e: 0.43
episode: 169/1000, score: -132.0, e: 0.43
episode: 170/1000, score: -47.80759975999899, e: 0.43
episode: 171/1000, score: -27.0, e: 0.43
episode: 172/1000, score: -905.0, e: 0.42
episode: 173/1000, score: -532.9476739500005, e: 0.42
episode: 174/1000, score: -5.0, e: 0.42
episode: 175/1000, score: -168.609881219998, e: 0.42
episode: 176/1000, score: -1431.5450504799992, e: 0.42
episode: 177/1000, score: -477.0, e: 0.41
episode: 178/1000, score: 3397.160477130001, e: 0.41
episode: 179/1000, score: -342.9021710199995, e: 0.41
episode: 180/1000, score: -526.8708424899978, e: 0.41
episode: 181/1000, score: 3722.6805673399977, e: 0.41
episode: 182/1000, score: -56.433678090001195, e: 0.4
episode: 183/1000, score: 3107.6241121899966, e: 0.4
episode: 184/1000, score: -167.10154865000004, e: 0.4
episode: 185/1000, score: -686.0, e: 0.4
episode: 186/1000, score: -346.65725

KeyboardInterrupt: 

def calc_next_state(state):
#Calc Capital (action == 0 then sell, action == 1 then buy) 5 coins * state_close_price
        #Calc position (current position -/+ differ from action)
        #Get newest state item and append capital and position
        #Remove latest entry and add new state item